In [1]:
import cv2
import cvzone
import numpy as np
import math
import datetime
from cvzone.ColorModule import  ColorFinder
cap = cv2.VideoCapture('Hackathon video.mov')
myColorFinder =  ColorFinder(False)
hsvVals={'hmin': 21, 'smin': 0, 'vmin': 0, 'hmax': 89, 'smax': 255, 'vmax': 255}
poslist=[]
pointsList = []
totalframecount= int(cap.get(cv2.CAP_PROP_FRAME_COUNT))


def gradient(pt1,pt2):
    if (pt1[0]-pt2[0])==0:
        return (pt1[1]-pt2[1])
    else:
        return (pt1[1]-pt2[1])/(pt1[0]-pt2[0])
def getAngle(pt1,pt2,pt3):
    m1 = gradient(pt1,pt2)
    m2 = gradient(pt2,pt3)
    if 1+(m2*m1)!=0:
        angR = math.atan((m2-m1)/(1+(m2*m1)))
    else:
        angR = math.atan(m2-m1)
    angD = round(math.degrees(angR))
    return angD
Angle=[]
pitch=[]
bounce_count=1
start_time=datetime.datetime.now()
start_time=start_time.timestamp()
time_list=[]
quadrant=[]
bounce_number=[]
time_bounce=[]
frame_number=[]

while True:
    success, img=cap.read()
    try:
        img = cv2.resize(img, (1400, 1000), interpolation=cv2.INTER_AREA)
        
    except:
        break
    #img = cv2.resize(img, (0, 0), None, 0.35, 0.35)
    imgcolor, mask= myColorFinder.update(img,hsvVals)
    imgContours,contours=cvzone.findContours(img,mask,minArea=500)
    area=[(15,250),(430,20),(900,155),((540,550))]
    quad1=[(15,250),(250,115),(470,200),(230,370)]
    quad2=[(250,115),(430,20),(630,75),(470,200)]
    quad3=[(230,370),(470,200),(750,310),(540,550)]
    quad4=[(470,200),(630,75),(900,155),(750,310)]
    if contours:
        current_time=datetime.datetime.now()
        current_time=current_time.timestamp()
        lapsed_time=(current_time-start_time)
       
        time_list.append(round(lapsed_time,3))
        poslist.append(contours[0]['center'])
        cv2.circle(imgContours,(contours[0]['center']),5,(0,255,0),cv2.FILLED)
        if len(poslist)>2:
            a=getAngle(tuple(poslist[-3]),tuple(poslist[-2]),tuple(poslist[-1]))
            Angle.append(a)
            cv2.circle(imgContours,(poslist[-3]),5,(255,0,255),cv2.FILLED)
            cv2.circle(imgContours,(poslist[-2]),5,(255,0,255),cv2.FILLED)
            cv2.circle(imgContours,(poslist[-1]),5,(255,0,255),cv2.FILLED)    
    else:
        val=0
        if len(Angle)>1:
            maxi=max(Angle)
            i=Angle.index(maxi)
            if Angle[i]==Angle[i+1]:
                i=i+1
            bounce_number.append(bounce_count)
            print("Bounce=",bounce_count)
            if poslist[i+1]:
                pitch=poslist[i+1]
                pitch_time=time_list[i+1]
            else:
                pitch=poslist[i]
                pitch_time=time_list[i]
            
            
            #print("Time=",pitch_time)
            #print("Pitch point",pitch)
            (cx,cy)=pitch
            res=cv2.pointPolygonTest(np.array(quad1,np.int32),(int(cx),int(cy)),False)
            if res>=0:
                print("Quadrant 1")
                val=1
                cv2.polylines(imgContours,[np.array(quad1,np.int32)],True,(15,220,10),6)
                cv2.circle(imgContours,(cx,cy),5,(0,255,0),cv2.FILLED)
                a=int(cap.get(cv2.CAP_PROP_POS_FRAMES))
                print("frame",a)
                a1=(a/totalframecount)*21
                print("bounce_time",a1)
                frame_number.append(a)
                time_bounce.append(round(a1,3))
                quadrant.append("1")
            res=cv2.pointPolygonTest(np.array(quad2,np.int32),(int(cx),int(cy)),False)
            if res>=0:
                print("Quadrant 2")
                val=1
                cv2.polylines(imgContours,[np.array(quad2,np.int32)],True,(15,220,10),6)
                cv2.circle(imgContours,(cx,cy),5,(0,255,0),cv2.FILLED)
                b=int(cap.get(cv2.CAP_PROP_POS_FRAMES))
                b1=(b/totalframecount)*21
                print("bounce_time",b1)
                print("frame",b)
                frame_number.append(b)
                time_bounce.append(round(b1,3))
                quadrant.append("2")
     
            res=cv2.pointPolygonTest(np.array(quad3,np.int32),(int(cx),int(cy)),False)
            if res>=0:
                print("Quadrant 3")
                val=1
                cv2.polylines(imgContours,[np.array(quad3,np.int32)],True,(15,220,10),6)
                cv2.circle(imgContours,(cx,cy),5,(0,255,0),cv2.FILLED)
                c=int(cap.get(cv2.CAP_PROP_POS_FRAMES))
                c1=(c/totalframecount)*21
                print("bounce_time",c1)
                frame_number.append(c)
                quadrant.append("3")
                time_bounce.append(round(c1,3))
                print("frame",c)
            res=cv2.pointPolygonTest(np.array(quad4,np.int32),(int(cx),int(cy)),False)
            if res>=0:
                print("Quadrant 4")
                val=1
                cv2.polylines(imgContours,[np.array(quad4,np.int32)],True,(15,220,10),6)
                cv2.circle(imgContours,(cx,cy),5,(0,255,0),cv2.FILLED)
                d=int(cap.get(cv2.CAP_PROP_POS_FRAMES))
                d1=(d/totalframecount)*21
                print("bounce_time",d1)
                frame_number.append(d)
                time_bounce.append(round(d1,3))
                quadrant.append("4")
                print("frame",d)
            if val==0:
                print("Outside")
                cv2.polylines(imgContours,[np.array(area,np.int32)],True,(15,220,10),6)
                e=int(cap.get(cv2.CAP_PROP_POS_FRAMES))
                e1=(e/totalframecount)*21
                
                print("bounce_time",e1)
                time_bounce.append(round(e1,3))
                frame_number.append(e)
                quadrant.append("out_of_bound")
                print("frame",e)
            bounce_count+=1
            cv2.circle(imgContours,(pitch),5,(0,0,255),cv2.FILLED)
        poslist=[]
        Angle=[]
        time_list=[]
    
    cv2.imshow("Image", imgContours)
    cv2.waitKey(1)

cap.release()
cv2.destroyAllWindows()

Bounce= 1
Outside
bounce_time 0.8095987411487018
frame 49
Bounce= 2
Outside
bounce_time 1.6026750590086547
frame 97
Bounce= 3
Quadrant 3
bounce_time 2.957513768686074
frame 179
Bounce= 4
Quadrant 1
frame 325
bounce_time 5.36978756884343
Bounce= 5
Outside
bounce_time 6.377655389457121
frame 386
Bounce= 6
Quadrant 4
bounce_time 8.839496459480724
frame 535
Bounce= 7
Outside
bounce_time 12.011801730920535
frame 727
Bounce= 8
Outside
bounce_time 14.820613690007868
frame 897
Bounce= 9
Quadrant 1
frame 928
bounce_time 15.332808811959087
Bounce= 10
Outside
bounce_time 16.93548387096774
frame 1025
Bounce= 11
Quadrant 4
bounce_time 18.009441384736427
frame 1090
Bounce= 12
Quadrant 4
bounce_time 20.124311565696303
frame 1218


In [9]:
print(quadrant)

['out_of_bound', 'out_of_bound', '3', '1', 'out_of_bound', '4', 'out_of_bound', 'out_of_bound', '1', 'out_of_bound', '4', '4']


In [10]:
print(bounce_number)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


In [11]:
print(time_bounce)

[0.81, 1.603, 2.958, 5.37, 6.378, 8.839, 12.012, 14.821, 15.333, 16.935, 18.009, 20.124]


In [12]:
print(frame_number)

[49, 97, 179, 325, 386, 535, 727, 897, 928, 1025, 1090, 1218]


In [13]:
import pandas as pd

In [14]:
df=pd.DataFrame({
    "bounce_number":bounce_number,
    "time_of_bounce":time_bounce,
    "quadrant_of_bounce":quadrant,
    "frame_number":frame_number
})

df.to_csv("output.csv",index=False)

In [15]:
df.head()

,bounce_number,time_of_bounce,quadrant_of_bounce,frame_number
0,1,0.810,out_of_bound,49
1,2,1.603,out_of_bound,97
2,3,2.958,3,179
3,4,5.370,1,325
4,5,6.378,out_of_bound,386
